# ✏️ Basic Setting

## 🔎 Importing Libraries

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns; sns.set()
from tqdm import tqdm_notebook as tqdm
import numpy as np
import os
from glob import glob 
from sklearn.manifold import TSNE

from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_samples, silhouette_score
from sklearn.model_selection import train_test_split
from sklearn import preprocessing
import lightgbm as lgb
from sklearn.model_selection import PredefinedSplit
from sklearn.model_selection import GridSearchCV, ParameterGrid
from sklearn.metrics import (roc_curve, auc, accuracy_score)
from sklearn.metrics import roc_auc_score
from sklearn.metrics import f1_score
from xgboost import XGBClassifier, plot_importance
from sklearn import metrics
from sklearn.metrics import pairwise_distances
%matplotlib inline

pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 50)
pd.set_option('display.width', 1000)
pd.set_option('display.max_colwidth', 500)

from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

import glob
from IPython.display import display, HTML

import multiprocessing as mp
print('Number of CPU cores:', mp.cpu_count())

Number of CPU cores: 12


## 🔎 Data Load

In [ ]:
variable_dtype = pd.read_excel("../[Track1_데이터4] variable_dtype.xlsx")

In [ ]:
dict_data = {}
for data in glob('*.csv'):
    name = data.split('.')[0]
    print(name)
    dict_data[name] = pd.read_csv(data, encoding='euc-kr')

quiz
cst_feat_feb_quiz
val
cst_feat_jan
mrc_info
train


# ✏️ T-sne

## 🔎 T-sne reduction

In [ ]:
df_sort = df[new_column]
df_sort

In [ ]:
df_sort_not_zero = df_sort[df_sort['MRC_ID_DI'] != 0]

In [ ]:
df_sort_values = df_sort_not_zero.iloc[:,30:]

In [ ]:
distance_matrix_doc = pairwise_distances(df_sort_values, df_sort_values, metric='cosine', n_jobs=-1)
tsne_doc = TSNE(metric="precomputed", n_components=2, verbose=1, n_iter=500)
tsne_results_doc = tsne_doc.fit_transform(distance_matrix_doc)

In [ ]:
tsne_doc = pd.DataFrame()
tsne_doc['tsne-2d-one'] = tsne_results_doc[:,0]
tsne_doc['tsne-2d-two'] = tsne_results_doc[:,1]

In [ ]:
tsne_doc_concat = pd.concat([samp_cst_feat['cst_id_di'], tsne_doc], axis=1)
tsne_doc_concat = tsne_doc_concat.merge(samp_train, how="left", on="cst_id_di")

## 🔎 Visualization

In [ ]:
dict_df_class = {}
for i in range(1, 10, 2):
    dict_df_class[i] = df[df['MRC_ID_DI']==i][new_column]
    df_class_num_values = dict_df_class[i].iloc[:, 31:].values
    
    distance_matrix_doc = pairwise_distances(df_class_num_values, df_class_num_values, metric='cosine', n_jobs=-1)
    tsne_doc = TSNE(metric="precomputed", n_components=2, verbose=1, perplexity=30, n_iter=500)
    tsne_results_doc = tsne_doc.fit_transform(distance_matrix_doc)
    
    tsne_doc_df = pd.DataFrame()
    tsne_doc_df['tsne-2d-one'] = tsne_results_doc[:,0]
    tsne_doc_df['tsne-2d-two'] = tsne_results_doc[:,1]
    
    tsne_doc_concat = pd.concat([dict_df_class[i].reset_index().drop(columns=['index'])['cst_id_di'], tsne_doc_df.reset_index().drop(columns=['index'])], axis=1)
    tsne_doc_concat = tsne_doc_concat.merge(dict_df_class[i].iloc[:, :2], how="left", on="cst_id_di")
    
    plt.figure(figsize=(20, 20))
    sns.scatterplot(
        x="tsne-2d-one", y="tsne-2d-two",
        hue='MRC_ID_DI',
        palette=sns.color_palette("viridis", tsne_doc_concat['MRC_ID_DI'].nunique()),
        data=tsne_doc_concat,
        legend="full",
        alpha=1
    );
    print('{}번 클라스입니다.'.format(i))
    plt.show();